In [1]:
!pip install pyspark

## LogisticRegression

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()

In [4]:
data = spark.read.csv(path = "/kaggle/input/pyspark-ml-logistic-regression/titanic.csv",
                      inferSchema = True, header = True)

data.printSchema()

In [5]:
data = data.select(["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"])
data = data.na.drop()

In [6]:
data.show()

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer

In [8]:
gender_indexer = StringIndexer(inputCol = "Sex", outputCol = "SexIndex")
gender_encoder = OneHotEncoder(inputCol = "SexIndex", outputCol = "SexVec")

In [9]:
data = gender_indexer.fit(data).transform(data)
data = gender_encoder.fit(data).transform(data)

data.show(5)

In [10]:
embark_indexer = StringIndexer(inputCol = "Embarked", outputCol = "EmbarkIndex")
embark_encoder = OneHotEncoder(inputCol = "EmbarkIndex", outputCol = "EmbarkVec")

In [11]:
data = embark_indexer.fit(data).transform(data)
data = embark_encoder.fit(data).transform(data)

data.show(5)

In [12]:
assembler = VectorAssembler(inputCols = ["Pclass", "SexVec", "Age", "SibSp", "Parch", "Fare", "EmbarkVec"],
                            outputCol = "features")

In [13]:
output = assembler.transform(data)
output.show(5)

In [14]:
train, test = output.randomSplit(weights = [0.7, 0.3], seed = 42)

In [15]:
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(featuresCol = "features",
                             labelCol = "Survived",
                             predictionCol = "prediction")

In [16]:
model = log_reg.fit(train)

In [17]:
y_hat = model.evaluate(test)

y_hat.predictions.select("Survived", "prediction").show()

In [18]:
y_hat.areaUnderROC

### Pipeline

In [19]:
from pyspark.ml import Pipeline

data = spark.read.csv(path = "/kaggle/input/pyspark-ml-logistic-regression/titanic.csv",
                      inferSchema = True, header = True)

gender_indexer = StringIndexer(inputCol = "Sex", outputCol = "SexIndex")
gender_encoder = OneHotEncoder(inputCol = "SexIndex", outputCol = "SexVec")

embark_indexer = StringIndexer(inputCol = "Embarked", outputCol = "EmbarkIndex")
embark_encoder = OneHotEncoder(inputCol = "EmbarkIndex", outputCol = "EmbarkVec")

assembler = VectorAssembler(inputCols = ["Pclass", "SexVec", "Age", "SibSp", "Parch", "Fare", "EmbarkVec"],
                            outputCol = "features")

model = LogisticRegression(featuresCol = "features",
                           labelCol = "Survived",
                           predictionCol = "prediction")

In [20]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, model])

In [21]:
train, test = data.randomSplit(weights = [0.7, 0.3], seed = 42)

In [22]:
model = pipeline.fit(train)

In [ ]:
y_hat = model.transform(test)

In [ ]:
y_hat.select("Survived", "prediction").show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol = "prediction", labelCol = "Survived")

evaluator.metricName

# Por defecto la metrica es la curva ROC

In [ ]:
# Sin parametros de metricas
evaluator.evaluate(dataset = y_hat)

In [ ]:
# Con parametros
evaluator.evaluate(dataset = y_hat, params = {evaluator.metricName: "areaUnderROC"})

In [ ]:
# Area Under Precision-Recall, esta metrica puede resultar util cuando las clases estan desbalanceadas
evaluator.evaluate(dataset = y_hat, params = {evaluator.metricName: "areaUnderPR"})

In [ ]:
################################################################################################################################